# TDT4173 Project Report

Course: TDT4173 Modern Machine Learning in Practice

Year: 2024

Group name: [55] Ship Happens

Group members:
| Name     | StudentId |
| -------- | ------- |
| Live Krogh  | 123    |
| Ingvild Langeggen  | 123     |
| Alexander Sundquist    | 132896    |

## Table of Contents

 1. [Introduction](#1-introduction)
 2. [Exploratory data analysis](#2-exploratory-data-analysis)
 3. [Feature engineering](#3-feature-engineering)
 4. [Modeling](#4-modeling)
 5. [Model interpretation](#5-model-interpretation)

## 1 Introduction

In the following document the steps taken to complete the TDT4173 Modern Machine Learning in Practice project will be described.

### 1.1 Set-up

In [ ]:
import pandas as pd
from datetime import timedelta
import geopy.distance

The packages that were used in the project were:
- pandas
- datetime
- ...

In [ ]:
train = pd.read_csv('ais_train.csv', sep='|')
test = pd.read_csv('ais_test.csv', sep=',')
vessels = pd.read_csv('vessels.csv', sep='|')
ports = pd.read_csv('ports.csv', sep='|')

- The training dataset, `ais_train.csv`, was imported as the dataframe `train`. 
- The test dataset, `ais_test.csv`, was imported as the dataframe `test`. 
- The vessels dataset, `vessels.csv`, was imported as the dataframe `vessels`. 
- The ports dataset, `ports.csv`, was imported as the dataframe `ports`. 

## 2 Exploratory data analysis

Do at least 4 of the following:
- Search domain knowledge
- Check if the data is intuitive
- Understand how the data was generated
- Explore individual features
- Explore pairs and groups of features
- Clean up features
    - Timedate

### Search domain knowledge

### Check if the data is intuitive

### Understand how the data was generated

### Explore individual features

### Explore pairs and groups of features

### Clean up features

## 3 Feature engineering

**To avoid the deduction, you should show that you have tried one or more feature engineering techniques (i.e., feature selection and/or feature extraction) in the report.**

For the feature engineering, both feature selection and extraction were conducted. The feature engineering was initially based on the exploratory data analysis but was also adjusted iteratively based on prediction results.

The main function for feature engineering `feature_engineering_heading(test,train)` can be seen below. The function inputs the test dataframe and the train dataframe and returns the dataframe, `features` with the decired features. In the following parts the specific feature selection and extraction that was conducted will be described in more detail.

```python
def feature_engineering_heading(test, train):
    test = test.copy()
    train = train.copy()
    features = pd.DataFrame()

    # Add the columns vesselId, time, latitude, longitude, cog and sog to the features from train
    features['vesselId'] = train['vesselId']
    features['portId'] = train['portId']
    features['time'] = train['time']
    features['etaRaw'] = train['etaRaw']
    features['latitude'] = train['latitude']
    features['longitude'] = train['longitude']
    features['cog'] = train['cog']
    features['sog'] = train['sog']

    # Create a new binary column for 'under_way' based on the navstat column in train. 
    # 1 if navstat is 0 or 8, 0 otherwise
    features['under_way'] = train['navstat'].isin([0, 8]).astype(int)

    # Adding the columns latitude_1 (last seen latitude) and longitude_1 (last senn longitude)
    # to the features from train. For each vessel
    # Also adding time_1 (time difference from current time to last seen time)
    features['latitude_1'] = train.groupby('vesselId')['latitude'].shift(1)
    features['longitude_1'] = train.groupby('vesselId')['longitude'].shift(1)
    features['time_1'] = train.groupby('vesselId')['time'].diff().dt.total_seconds()

    # Adding the columns latitude_2 (second last seen latitude) and longitude_2 (second last seen longitude)
    # to the features from train. For each vessel
    # Also adding time_2 (time difference from current time to second last seen time)
    features['latitude_2'] = train.groupby('vesselId')['latitude'].shift(2)
    features['longitude_2'] = train.groupby('vesselId')['longitude'].shift(2)
    features['time_2'] = train.groupby('vesselId')['time'].diff(2).dt.total_seconds()

    # Adding the columns latitude_3 (third last seen latitude) and longitude_3 (third last seen longitude)
    # to the features from train. For each vessel
    # Also adding time_3 (time difference from current time to third last seen time)
    features['latitude_3'] = train.groupby('vesselId')['latitude'].shift(3)
    features['longitude_3'] = train.groupby('vesselId')['longitude'].shift(3)
    features['time_3'] = train.groupby('vesselId')['time'].diff(3).dt.total_seconds()

    # Apply the conversion function to create a new 'eta' column
    features['eta'] = features.apply(convert_eta_raw, axis=1)

    # Calculate the time difference between 'time' and 'eta' in seconds
    features['time_difference_eta'] = (features['eta'] - features['time']).dt.total_seconds()

    # Split the time column into month, day, hour, minute, and second columns
    features['month'] = train['time'].dt.month
    features['day'] = train['time'].dt.day
    features['hour'] = train['time'].dt.hour
    features['minute'] = train['time'].dt.minute

    # Drop the 'time' and 'etaRaw' columns as they are no longer needed
    features.drop(['time', 'etaRaw'], axis=1, inplace=True)
    features.drop(['eta'], axis=1, inplace=True)

    # Merge the features with the ports
    # Drop rows where portId is null/Nan
    features = features.dropna(subset=['portId'])
    features['portId'] = features['portId'].astype(int)
    features = merge_with_ports(features, ports)

    # Add the feature 'changed_port' to the features.
    # 1 if the vessel has changed port, 0 otherwise
    # features['changed_port'] = features['portId'] != features['portId'].shift(1)

    # Drop the portId column
    # features = features.drop('portId', axis=1)

    # Drop 

    return features
```

### 3.1 Feature selection

When conducting the feature selection, the dataframe `features` was created in the function `feature_engineering_heading(test,train)` where the desired features from the dataframes `train` and `ports` were appended.

#### 3.1.1 Selection from `train`

From `train`, the features `vesselId`, `portId`, `latitude`, `longitude`, `cog`, and `sog` were directly selected. 

The calendar features, `month`, `day`, `hour`, and `minute`, describing the time at which the data point was recorded were also included as seperate featuers. The year and second that were also included in the raw data were, however, left out. The year for all data was 2024 which means that there was no variability and no information to be gained from including the feature. Seconds were to granular to provide any information **(Is this why we did it? just said something lol)**

The features `heading` and `rot` from `train` were not selected as final features. `heading` was not selected due to its high correlation with `cog`, which was deemed to be a more accurate representation of vessel direction **(is this true?)**. `rot` was not included as a feature due to its inclusion reducing performance **(is this true?)**

The features `etaRaw` and `navstat` from `train` were not directly selected but instead used to generate new features, which is described in more detail in [3.2 Feature engineering](#3.2-feature-engineering). `etaRaw` was not selected directly because it proved to be more valuable as a relational feature to `time`. `navstat` was not selected due to it mainly describing two states, under way, given by the feature values 0 and 8 and not under way, given by the feature values 1 and 5.

#### 3.1.2 Selection from `ports`

From `ports`, the features `longitude` and `latitude` were selected, and called `portLongitude` and `portLatitude`. The motivation was to give the model an idea of the vessels final location.

### 3.2 Feature extraction

#### 3.2.1 `under_way`

As mentioned in [3.1.1](#3.1.1-selected-from-train) `navstat` was not selected, but instead used to generate a new feature called `under_way`. The new feature values were set to 1 if the `navstat` value was 0 or 8 and set to 0 in all other cases.

#### 3.2.2 Lag features

The autoregressive lag features `latitude_1` and `longitude_1` was simply the latitude and longitude of the current vessel one time stamp ago. `latitude_2` and `longitude_2` as well as `latitude_3` and `longitude_3` were similirly lag features but from 2 and 3 time steps before, respectively. Lag features were described as beneficial during the lecture on time series and was therefore used in the project. 

### 3.2.3 Time since last seen

Furtermore, in combination with the lag features, the features `time_1`, `time_2`, and `time_3` were generated. The time features describe the time difference from current time to last, second last, and third last seen time. These features describe when the three lagged latitude and longitude features were recorded. This was done because sometimes there could be a big time difference between two following measurements. Intuitively, if previously seen latitude and longitude were a long time ago and the vessel has moved, the data should be less valuable in predicting the next position. The time features allow the model to interpret this pattern. 

#### 3.2.4 Time to arrival

The time to arrival, represented in the feature `time_difference_eta`, was extracted by subtracting the eta with the current time. Firstly, however, the feature `etaRaw` in `train` had to be preprocessed to allow for the feature extraction. `etaRaw` had to be converted to the datetime format and also had to be handled if the `etaRaw` value was earlier than `time` value of when the data was recorded. The processing was done by the function `convert_eta_raw(row)` as seen below. 

```python
# Function to convert 'etaRaw' to datetime format, handling year rollovers
def convert_eta_raw(row):
    current_year = row['time'].year  # Get the year from the 'time' column
    eta_with_year = f"{current_year}-{row['etaRaw']}"  # Create full datetime string
    eta_datetime = pd.to_datetime(eta_with_year, format='%Y-%m-%d %H:%M', errors='coerce')
    
    # Adjust year if 'etaRaw' is earlier than 'time'
    if eta_datetime < row['time']:
        eta_datetime = eta_datetime + timedelta(days=365)
    return eta_datetime
```

#### 3.2.5 Distance to port

## 4 Modeling

**To avoid the deduction, you should show that you have tried two or more types of predictors in the report (e.g., XGBoost and Random Forest). It is allowed to use only one type of predictor in a short notebook for Kaggle predictions.**

During the data preprocessing and feature engineerng process, the XGBoostRegressor model was used to validate iterations due to its quick run time. When a desirable preprocessing and feature engineering had been completed, a model testing phase was initiated. During testing two types of predictors were tested: XGBoostRegressor and **_____**. The best performing model was selected as the final model. Lastly, hyper-parameter tuning was conducted for the selected model.

### 4.1 Model testing

#### 4.1.1 XGBoost

#### 4.1.2 Other model

### 4.2 Final model

#### 4.2.1 Hyper-parameter tuning

## 5 Model interpretation

To avoid the deduction, you should show one or more model interpretation results (e.g., PDP, feature importance, LIME) in the report.